In [1]:
import requests
import string
import random
import datetime
import os
import json
import re
import pandas as pd

In [2]:
URL = "http://localhost:8080"

In [3]:

def login():
    url = URL + "/api/me"
    session = requests.Session()
    try:
        session.auth = ('admin', 'district')
        return session
    except:
        print("bad response")

def get_data(key):
    url = URL + "/api/me"
    session = requests.Session()
    session.auth = ('admin', 'district')
    response = session.get(URL + "/api/" + key )
    if response.status_code == 200:
        json_response = json.loads(response.text)
        return json_response
    else:
        print("bad response: " + str(response.status_code))
        return None

In [4]:
def getOrgUnitInfoByLevel(name):
    """
    :param name: name used to filter in to OrgUnits
    :return: This method return the basic information about specific organisation unit filtered by id
    """
    try:
        session = login()
        url = f"http://localhost:8080/api/organisationUnits.json?"
        params = {
            "filter": f"level:like:{name}"
        }
        response = session.get(url, headers={'content-type': 'application/json'}, params=params)
        print(f"[FETCHING ORG-UNIT INFO] >> {response.text}")
    except Exception as e:
        print("Unspected exception")
        print(e)

In [5]:
response = get_data("organisationUnits?level=3")
orgUnits = pd.DataFrame(response["organisationUnits"])

In [6]:
orgUnits

,displayName,id
0,Adeje,QMw3vvM0Hc4
1,Adrar,n7tAJt9hHtX
2,Agaete,wksUKHz27kj
3,Agulo,xFSKshO8IWE
4,Agüimes,lGHoWxCXAee
5,Alajeró,ALrCsdgoUFd
6,Antigua,ce8SMdF9fAb
7,Arafo,iPsPhnkjFnF
8,Arico,GMVSwpw0J5F
9,Arona,DAgKPHgHJGi


In [7]:
def createOrgUnit(parentOrgUnit,name,preriodType:str):
    """
    :param code:
    :param name:
    :param shortname:
    :param preriodType:
    :return:
    """

    """parent its Sierra leone instance or main instance of organization unit"""
    payload = {
        'name': name,
        'shortName': name,
        'periodType': preriodType,
        'openingDate': "2020-01-01",
        "parent": {
            "id": parentOrgUnit["id"],
        }
    }

    session = login()
    url = "http://localhost:8080/api/organisationUnits"    
    response = session.post(url, json=payload, headers={'content-type': 'application/json'})
    if response.status_code == 200:
        print((f"[CREATING ORG-UNIT] >> {response.text}"))
    else:
        print("bad response: " + str(response.status_code) + response.text)
        return response

In [8]:
orgUnits.head()

,displayName,id
0,Adeje,QMw3vvM0Hc4
1,Adrar,n7tAJt9hHtX
2,Agaete,wksUKHz27kj
3,Agulo,xFSKshO8IWE
4,Agüimes,lGHoWxCXAee


In [9]:
def create_facility(OrgUnitsdf,name:str,preriodType:str):
    for i in range(len(OrgUnitsdf)):
        full_name = name +"_" + OrgUnitsdf.loc[i]["displayName"]
        createOrgUnit(OrgUnitsdf.loc[i],full_name,preriodType)

In [10]:
# no los crea en todos.. no sé porqué?
response = create_facility(orgUnits,"sanitary_center","Weekly")

bad response: 201{"httpStatus":"Created","httpStatusCode":201,"status":"OK","response":{"responseType":"ObjectReport","uid":"tR1ESmLTDC1","klass":"org.hisp.dhis.organisationunit.OrganisationUnit","errorReports":[]}}
bad response: 201{"httpStatus":"Created","httpStatusCode":201,"status":"OK","response":{"responseType":"ObjectReport","uid":"R21VOJyasOa","klass":"org.hisp.dhis.organisationunit.OrganisationUnit","errorReports":[]}}
bad response: 201{"httpStatus":"Created","httpStatusCode":201,"status":"OK","response":{"responseType":"ObjectReport","uid":"mLjDSrWnUnt","klass":"org.hisp.dhis.organisationunit.OrganisationUnit","errorReports":[]}}
bad response: 201{"httpStatus":"Created","httpStatusCode":201,"status":"OK","response":{"responseType":"ObjectReport","uid":"TJBXrFiUbsr","klass":"org.hisp.dhis.organisationunit.OrganisationUnit","errorReports":[]}}
bad response: 201{"httpStatus":"Created","httpStatusCode":201,"status":"OK","response":{"responseType":"ObjectReport","uid":"K5AUrwTfmF6

In [11]:
response = create_facility(orgUnits,"Hospital","Weekly")


bad response: 201{"httpStatus":"Created","httpStatusCode":201,"status":"OK","response":{"responseType":"ObjectReport","uid":"K2sYFAy8NAo","klass":"org.hisp.dhis.organisationunit.OrganisationUnit","errorReports":[]}}
bad response: 201{"httpStatus":"Created","httpStatusCode":201,"status":"OK","response":{"responseType":"ObjectReport","uid":"ppkwO83EQzm","klass":"org.hisp.dhis.organisationunit.OrganisationUnit","errorReports":[]}}
bad response: 201{"httpStatus":"Created","httpStatusCode":201,"status":"OK","response":{"responseType":"ObjectReport","uid":"ib4dMYZzKNU","klass":"org.hisp.dhis.organisationunit.OrganisationUnit","errorReports":[]}}
bad response: 201{"httpStatus":"Created","httpStatusCode":201,"status":"OK","response":{"responseType":"ObjectReport","uid":"GkWlDtDTZi8","klass":"org.hisp.dhis.organisationunit.OrganisationUnit","errorReports":[]}}
bad response: 201{"httpStatus":"Created","httpStatusCode":201,"status":"OK","response":{"responseType":"ObjectReport","uid":"PEDEu3X1bAu